In [1]:
from ThoughtKit import *
import sys
sys.modules['ThoughtKit'].DEBUG = True
sys.modules['ThoughtKit'].MODEL = "gpt-4o-mini"
from pprint import pprint

In [6]:
query = """You are playing Russian roulette with a six-shooter revolver. 
Your opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. 
He gives you the choice of whether or not he should spin the chambers again before firing at you. 
Should he spin again? Your final answer should start with yes or no."""
tree = Tree(Status_model, Thought_model, Solution_model, Equivalence_model, Compare_model, max_depth=5)
tree._max_num_solutions = 4
thoughts, best = tree.shortest_paths(query, max_paths_per_node = 5)

Current Depth: 0 out of 5


In [3]:
pprint(best)

'no'


In [4]:
tree.solutions

['no']

In [5]:
tree.solution_thoughts

[['You are playing Russian roulette with a six-shooter revolver. \nYour opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. \nHe gives you the choice of whether or not he should spin the chambers again before firing at you. \nShould he spin again? Your final answer should start with yes or no.',
  'If the opponent fired and no bullet came out, there is one empty chamber and five chambers with bullets. The probability of the next chamber being empty without spinning again is 1 out of 5, while spinning again gives a 1 out of 6 chance of being empty. I need to compare these probabilities to make a decision on whether he should spin again or not.',
  'The probability of the next chamber being empty without spinning again is 1/5, while the probability of it being empty if he spins again is 1/6. I need to determine which probability is higher to decide if he should spin again or not.',
  'Since 1/5 is greater than 1/6, it is safer to not spin agai

In [72]:
sys.modules['ThoughtKit'].DEBUG = False

from tqdm import tqdm

aneal_tree_results = []
bin_tree_results = []
chain_results = []
anneal_chain_results = []
regular_tree_results = []

aneal_chain = AnnealChain(Status_model, Thought_model, Solution_model)
chain = Chain(Status_model, Thought_model, Solution_model)
bintree = BinTree(Status_model, Thought_model, Solution_model, Equivalence_model, Compare_model, max_depth=5)
anneal_tree = SimpleAnnealTree(Status_model, Thought_model, Solution_model, Equivalence_model, Compare_model, max_depth=5)
tree = Tree(Status_model, Thought_model, Solution_model, Equivalence_model, Compare_model, max_depth=5)

question = """You are playing Russian roulette with a six-shooter revolver. 
Your opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. 
He gives you the choice of whether or not he should spin the chambers again before firing at you. 
Should he spin again? Your final answer should start with a yes or no."""#"If I am not not not not not hungry, do I want to eat? Your final answer should be a yes or no."

for i in tqdm(range(20)):
    
    try:
        best = aneal_chain(question, T_init=1.2, max_length=5)
        anneal_chain_results.append(best)
    except:
        pass
    try:
        best = chain(question, max_length=5, T=1.2)
        chain_results.append(best)
    except:
        pass
    try:
        thoughts, best = bintree(question)
        bin_tree_results.append(best)
    except:
        pass
    try:
        thoughts, best = anneal_tree(question, T_init=1.2)
        aneal_tree_results.append(best)
    except:
        pass
    try:
        thoughts, best = tree(question, T=1.2)
        regular_tree_results.append(best)
    except:
        pass

100%|██████████| 20/20 [46:33<00:00, 139.68s/it] 


In [34]:
count_nos = lambda l: sum([1 for i in l if "yes" in i[:5].lower()])/len(l)

print(f"Anneal Chain: {count_nos(anneal_chain_results)} \n Chain: {count_nos(chain_results)} \n regular tree: {count_nos(regular_tree_results)} \n Bin Tree: {count_nos(bin_tree_results)} \n Anneal Tree: {count_nos(aneal_tree_results)}")

NameError: name 'anneal_chain_results' is not defined

In [71]:
anneal_tree.solution_thoughts

[['You are playing Russian roulette with a six-shooter revolver. \nYour opponent puts in five bullets, spins the chambers and fires at himself, but no bullet comes out. \nHe gives you the choice of whether or not he should spin the chambers again before firing at you. \nShould he spin again? Your final answer should be a yes or no.',
  "To determine whether spinning again is safer, I need to assess the current situation and probabilities without moving Chambers, given one safe shot has occurred already, compared to the distribution of chambers post-spin. Calculating the likelihood of getting shot if he doesn't spin again versus if he does will guide my decision.",
  'If the opponent does not spin again, there are only five possible chambers remaining, one of which is empty. Since he survived the first shot, we know that one of the chambers was not selected, leaving four bullets and one empty remaining in the five available chambers. I will calculate the probability of survival if the c

In [74]:
regular_tree_results

['Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes']